In [34]:
import numpy as np
from utils import load_object

In [53]:
train = load_object('Data/Walker2d-v2/test_resumed/seed0/eval.pkl')

In [52]:
train

array([[ 4.0000000e+00,  1.1400000e+02,  0.0000000e+00, -4.4939013e+00],
       [ 9.0000000e+00,  2.7800000e+02,  2.1000000e+01,  3.0493269e+00],
       [ 1.4000000e+01,  4.1300000e+02,  1.5600000e+02,  2.6260724e+02],
       [ 1.9000000e+01,  5.4300000e+02,  2.8600000e+02,  2.6676154e+02],
       [ 2.4000000e+01,  6.3200000e+02,  3.7500000e+02,  2.5351755e+02],
       [ 2.9000000e+01,  7.0700000e+02,  4.5000000e+02,  2.5351877e+02],
       [ 3.4000000e+01,  7.8900000e+02,  5.3200000e+02,  2.8674121e+02],
       [ 3.9000000e+01,  9.1700000e+02,  6.6000000e+02,  2.7816992e+02],
       [ 4.4000000e+01,  1.2650000e+03,  1.0080000e+03,  2.5804855e+02],
       [ 4.9000000e+01,  1.9980000e+03,  1.7410000e+03,  2.9380148e+02],
       [ 5.4000000e+01,  2.8850000e+03,  2.6280000e+03,  2.5648511e+02],
       [ 5.9000000e+01,  3.7900000e+03,  3.5330000e+03,  1.2808992e+02]],
      dtype=float32)

In [54]:
train

array([[ 4.0000000e+00,  1.1400000e+02,  0.0000000e+00, -4.4939013e+00],
       [ 9.0000000e+00,  2.7800000e+02,  2.1000000e+01,  3.0493269e+00],
       [ 1.4000000e+01,  4.1300000e+02,  1.5600000e+02,  2.6260724e+02],
       [ 1.9000000e+01,  5.4300000e+02,  2.8600000e+02,  2.6676154e+02],
       [ 2.4000000e+01,  6.3200000e+02,  3.7500000e+02,  2.5351755e+02],
       [ 2.9000000e+01,  7.0700000e+02,  4.5000000e+02,  2.5351877e+02],
       [ 3.4000000e+01,  7.8900000e+02,  5.3200000e+02,  2.8674121e+02],
       [ 3.9000000e+01,  9.1700000e+02,  6.6000000e+02,  2.7816992e+02],
       [ 4.4000000e+01,  1.2650000e+03,  1.0080000e+03,  2.5804855e+02],
       [ 4.9000000e+01,  1.9980000e+03,  1.7410000e+03,  2.9380148e+02],
       [ 5.4000000e+01,  2.8850000e+03,  2.6280000e+03,  2.5648511e+02],
       [ 5.9000000e+01,  3.7900000e+03,  3.5330000e+03,  1.2808992e+02],
       [ 6.4000000e+01,  4.6690000e+03,  4.4120000e+03,  2.9065054e+02]],
      dtype=float32)